<a href="https://colab.research.google.com/github/aansheeagrwal/Deep-Learning/blob/main/prefetch_caching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__


'2.18.0'

In [3]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )


In [4]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [5]:
%%timeit
benchmark(FileDataset())

283 ms ± 17.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
benchmark(FileDataset().prefetch(1))


310 ms ± 66.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

313 ms ± 65.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
dataset = tf.data.Dataset.range(5)
dataset = dataset.map(lambda x: x**2)
dataset = dataset.cache("mycache.txt")
# The first time reading through the data will generate the data using
# `range` and `map`.
list(dataset.as_numpy_iterator())


[np.int64(0), np.int64(1), np.int64(4), np.int64(9), np.int64(16)]

In [10]:
# Subsequent iterations read from the cache.
list(dataset.as_numpy_iterator())

[np.int64(0), np.int64(1), np.int64(4), np.int64(9), np.int64(16)]

In [11]:
def mapped_function(s):
    # Do some hard pre-processing
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [12]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function), 5)

1.43 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%%timeit -r1 -n1
benchmark(FileDataset().map(mapped_function).cache(), 5)

460 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


further reading https://www.tensorflow.org/guide/data_performance#caching